# Zeitepochen im langen 19ten Jahrhundert

Eric Hobsbawm :
das Zeitalter der Revolution (1789–1848)
das Zeitalter des Kapitals (1848–1875) 
das Zeitalter des Imperiums (1875–1914)

[3] Reinhart Koselleck prägte den Begriff der Sattelzeit, die etwa von 1770 bis 1830 gedauert habe.

In [1]:
import sys
sys.path.append('../lib')
sys.path.append('../src')
sys.path.append('../src/queries')
sys.path.append('../src/clustering')
sys.path.append('../src/visualization_lib')
sys.path.append('../data')
sys.path.append('../')

%matplotlib inline


from lib import csv_reader as reader
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math

from lib import year_classifier as year_classifier

from jinja2 import Environment, FileSystemLoader

In [63]:
#merge Data!!!
import db
import json

def append_mid_year(df, column='mid_year'):
    '''
    Adds a int column (default "mid_year") to the table. It is the mean (rounded up) of from_year and to_year.
    '''
    df[column] = (df['from_year']+df['to_year'])/2
    df[column] = (df['mid_year']+0.49).astype(int)
    return df

def append_cluster_name(df, column='cluster_name'):
    # TODO!!!
    with open('../data/topics_benannt.txt') as topic_word_json:
        topic_word_dic = json.load(topic_word_json)
    
    df['cluster_name'] = topic_word_dic[str(df['cluster_id'][1].astype(str))]
    return df
    

my_db = db.Db()



# merge Tags
meta_tag_120_df = pd.merge(my_db.metadata_long_19, my_db.clusters_long_19)
#meta_tag_120_df['clusters_count'] = 4

#append_cluster_name(meta_tag_120_df)



topics_per_picture = 4

append_mid_year(meta_tag_120_df);

print ('-> merging clusters_long_19 lost %d that is %f p.' % (len(my_db.metadata_long_19) - len(meta_tag_120_df)/4, float(len(meta_tag_120_df)) / len(my_db.metadata_long_19)/4))
# pictures with tags only!
#meta_tag_120_df

# merge artists 
# on picture_id
meta_tag_120_artists_df = pd.merge(meta_tag_120_df, my_db.artist_origin[['picture_id','metadata_nationality','metadata_country','metadata_capital','metadata_longitude', 'metadata_latitude']], on='picture_id')

print ('-> merging artists lost %d that is %f p.' % (len(meta_tag_120_df)/topics_per_picture - len(meta_tag_120_artists_df)/topics_per_picture, float(len(meta_tag_120_artists_df)/topics_per_picture) / len(meta_tag_120_df)/topics_per_picture))


# merge picture titles
titles_df = my_db.artigo_titles.drop_duplicates(subset='picture_id')
meta_tag_120_artists_df = pd.merge(meta_tag_120_artists_df, titles_df, how='inner', left_index=True, on='picture_id')

print ('-> working with %d pictures.' % (len(meta_tag_120_artists_df)/4, ))






Db initialized, data is now accessible. It took  0.58s reading all files
-> merging clusters_long_19 lost 19194 that is 0.493000 p.
-> merging artists lost 3763 that is 0.049899 p.
-> working with 14901 pictures.


In [10]:
# matrix, topics_frequently_df


#u jedem untersuchendem topic ein df
# 'topic_id', 'name', 'count'

import json
with open('../data/topics_benannt.txt') as topic_word_json:
        topic_word_dic = json.load(topic_word_json)

        
        
        
topics = range(0,110)
#opics = [3,4]


matrixAll = [None] * 110
topics_frequently_df = [None]*110

for topicA in topics:
    #rint(topicA)
    
    picture_id_with_topic = meta_tag_120_artists_df[meta_tag_120_artists_df['cluster_id']==topicA]['picture_id']
    #for topicB in 
    a = meta_tag_120_artists_df[meta_tag_120_artists_df['cluster_id'] != topicA]

    matrix= [0] * 110

    for pic_id in picture_id_with_topic:
        pictures = a[a['picture_id']== pic_id]
        for topic_id in pictures['cluster_id']:
            matrix[topic_id] = matrix[topic_id] +1

    matrixAll[topicA] = matrix
    
    all_dfs = []
    for i, count in enumerate(matrix):
        df = pd.DataFrame([[i, topic_word_dic[str(i)], count]])
        df.columns= ['topic_id', 'name', 'count']
        all_dfs.append(df)

        
    # topics_frequently_df
    topics_frequently_df[topicA] = all_dfs[topics[0]]
    topics_frequently_df[topicA].columns= ['topic_id', 'name', 'count']
    for x in all_dfs[1:]:
        topics_frequently_df[topicA] = topics_frequently_df[topicA].append(x, ignore_index=True)

# results:
        
# matrix 
topic_martix_df = pd.DataFrame(matrixAll)

# topics_frequently_df
topics_frequently_df = topics_frequently_df

In [4]:
def plotMatrix(df_o):
    # Display a random matrix with a specified figure number and a grayscale
    # colormap
    #plt.clf() 
    
    df = df_o.copy(deep=True)
    
    #plt.clf()
    #plt.figure()
    
    plt.title('Topic Matrix')
    #plt.matshow(df.as_matrix(), fignum=100, cmap=plt.cm.gray)
    plt.matshow(df, cmap=plt.cm.gray)

    #topic_martix_path = dir_reports+"/plots/"+"topic_martix.jpg"
    plt.show()

    #return plt.gcf()

import numpy as np
import pandas as pd
from src.queries import db
from itertools import chain, combinations
import json


def frequency_matrix(dimension=110):
    
    
    with open('../data/topics_benannt.txt') as topic_word_json:
        topic_word_dic = json.load(topic_word_json)
    
    new_topics_frequently_df = [None]*110
    matrix = np.zeros(shape=(dimension,dimension), dtype=np.int)
    clusters = meta_tag_120_artists_df
    cluster_list = clusters.groupby('picture_id')['cluster_id'].apply(list)
    cluster_list = np.array(cluster_list)
    
    for transaction in cluster_list:
        transaction = sorted(transaction)
        #print(transaction)
        
        for z in chain.from_iterable(combinations(transaction, r) for r in range(len(transaction)+1)):
            if(len(z) == 2):
                #print('SET: {}'.format(z))
                i, j = z
                #print('z: {0}'.format(z))
                #print('i: {0}, j: {1}'.format(i,j))
                matrix[i,j] += 1
                matrix[j,i] += 1
                
    #new_topics_frequently_df
    new_topics_frequently_df = []
    for i in range(dimension):
        count = matrix[i]
        # TODO
        df = pd.DataFrame({'name': topic_word_dic, 'count':count})
        new_topics_frequently_df.append(df)

        
  
        
    return matrix, new_topics_frequently_df



m = frequency_matrix(110)


m_df = pd.DataFrame(m[0])
#plotMatrix(m_df)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [122]:
with open('../data/topics_benannt.txt') as topic_word_json:
        topic_word_dic = json.load(topic_word_json)

aa =[]
for i in range(0,110
    aa.append(topic_word_dic.get(str(i)))
aa

SyntaxError: invalid syntax (<ipython-input-122-d13ac63497c5>, line 6)

In [107]:
m[1][3]

,count,name
0,23,"Uebung, Goetter (siehe 33)"
1,1,"Grundriss, Entwurf"
2,6,"Antike Architektur, Saeulen"
3,0,"Gebirge, Landschaft"
4,9,"Asiatisch, japanisch"
5,136,"Uniform, Militaer"
6,31,"Utensilien, Faecher, Fernglas, Kimono"
7,28,"Wald, Baeume"
8,27,"Segelboot, Segelschiff"
9,320,"Boote, Schiffe"


In [83]:
m_df

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
0,0,3,12,23,45,21,28,19,2,6,...,84,0,2,5,7,45,5,7,5,123
1,0,0,3,1,4,43,1,7,96,1,...,0,46,3,100,119,9,1,146,3,1
2,0,0,0,6,4,173,5,21,14,1,...,5,1,2,4,2,12,2,156,2,8
3,0,0,0,0,9,136,31,28,27,320,...,1,0,34,9,115,16,4,22,13,44
4,0,0,0,0,0,9,9,4,1,3,...,41,0,4,0,3,23,7,19,5,109
5,0,0,0,0,0,0,56,109,29,40,...,5,8,5,37,90,13,5,7,12,18
6,0,0,0,0,0,0,0,18,1,7,...,2,0,7,9,1,23,4,23,5,8
7,0,0,0,0,0,0,0,0,7,18,...,35,2,5,3,28,25,5,42,2,23
8,0,0,0,0,0,0,0,0,0,19,...,2,18,7,43,74,1,8,107,10,6
9,0,0,0,0,0,0,0,0,0,0,...,0,2,19,3,77,9,1,12,5,20


In [86]:
topics_frequently_df[3]

,topic_id,name,count
0,0,"Wetter, Himmel",23
1,1,"Mann, elegante Kleidung",1
2,2,"Architektur, Fassade",6
3,3,"Frau, nackt",0
4,4,"Boot, Schiff",9
5,5,"Zeichnung, Skizze",136
6,6,Pferd,31
7,7,"Geometrische Formen, abstrakt",28
8,8,Sitzmoeglichkeit,27
9,9,"Frau, nackt, nah",320


In [ ]:
new_topics_frequently_df = [None]*110
for i, topic in enumerate(m_df):
    clusters = meta_tag_120_artists_df
    cluster_list = clusters.groupby('picture_id')['cluster_id'].apply(list)
    for x in cluster_list:
        new_topics_frequently_df[i] = 

In [84]:
clusters = meta_tag_120_artists_df
cluster_list = clusters.groupby('picture_id')['cluster_id'].apply(list)
cluster_list

picture_id
24              [37, 62, 103, 55]
25               [78, 21, 16, 57]
41                [21, 78, 6, 37]
44               [93, 8, 103, 69]
45                [53, 36, 26, 3]
46                [21, 8, 90, 84]
49                [53, 3, 26, 62]
50               [62, 93, 39, 21]
56               [84, 97, 11, 79]
57               [21, 103, 69, 5]
58                 [21, 8, 64, 5]
65               [108, 19, 81, 9]
70                 [8, 5, 11, 97]
71              [81, 106, 59, 34]
76               [48, 69, 97, 95]
77               [48, 49, 17, 79]
80              [106, 22, 31, 45]
82               [18, 3, 108, 68]
83               [62, 91, 64, 42]
92               [93, 90, 14, 65]
98               [18, 108, 45, 3]
99                [18, 45, 3, 68]
100               [18, 3, 45, 70]
101               [18, 3, 55, 23]
102             [3, 104, 109, 79]
103              [27, 95, 35, 37]
104              [83, 95, 51, 50]
106              [27, 95, 23, 35]
107              [12, 62, 14, 10]
108